# Config

In [32]:
# Must be encrypted
url = "http://127.0.0.1:8086"
token = "2WM######################################################RcbVQ=="
org = "team_keanu"
bucket = "testbucket"

# Write API

In [11]:
from datetime import datetime, timedelta

import pandas as pd
import rx
from pytz import UTC
from rx import operators as ops

from influxdb_client import InfluxDBClient, Point, WriteOptions

with InfluxDBClient(url=url, token=token, org=org) as _client:

    with _client.write_api(write_options=WriteOptions(batch_size=500,
                                                      flush_interval=10_000,
                                                      jitter_interval=2_000,
                                                      retry_interval=5_000,
                                                      max_retries=5,
                                                      max_retry_delay=30_000,
                                                      exponential_base=2)) as _write_client:

        """
        Write Line Protocol formatted as string
        """
        _write_client.write(bucket, org, "h2o_feet,location=coyote_creek water_level=1.0 1")
        _write_client.write(bucket, org, ["h2o_feet,location=coyote_creek water_level=2.0 2",
                                                    "h2o_feet,location=coyote_creek water_level=3.0 3"])

        """
        Write Line Protocol formatted as byte array
        """
        _write_client.write(bucket, org, "h2o_feet,location=coyote_creek water_level=1.0 1".encode())
        _write_client.write(bucket, org, ["h2o_feet,location=coyote_creek water_level=2.0 2".encode(),
                                                    "h2o_feet,location=coyote_creek water_level=3.0 3".encode()])

        """
        Write Dictionary-style object
        """
        _write_client.write(bucket, org, {"measurement": "h2o_feet", "tags": {"location": "coyote_creek"},
                                                    "fields": {"water_level": 1.0}, "time": 1})
        _write_client.write(bucket, org, [{"measurement": "h2o_feet", "tags": {"location": "coyote_creek"},
                                                     "fields": {"water_level": 2.0}, "time": 2},
                                                    {"measurement": "h2o_feet", "tags": {"location": "coyote_creek"},
                                                     "fields": {"water_level": 3.0}, "time": 3}])

        """
        Write Data Point
        """
        _write_client.write(bucket, org,
                            Point("h2o_feet").tag("location", "coyote_creek").field("water_level", 4.0).time(4))
        _write_client.write(bucket, org,
                            [Point("h2o_feet").tag("location", "coyote_creek").field("water_level", 5.0).time(5),
                             Point("h2o_feet").tag("location", "coyote_creek").field("water_level", 6.0).time(6)])

        """
        Write Observable stream
        """
        _data = rx \
            .range(7, 11) \
            .pipe(ops.map(lambda i: "h2o_feet,location=coyote_creek water_level={0}.0 {0}".format(i)))

        _write_client.write(bucket, org, _data)

        """
        Write Pandas DataFrame
        """
        _now = datetime.now(UTC)
        _data_frame = pd.DataFrame(data=[["coyote_creek", 1.0], ["coyote_creek", 2.0]],
                                   index=[_now, _now + timedelta(hours=1)],
                                   columns=["location", "water_level"])

        _write_client.write(bucket, org, record=_data_frame, data_frame_measurement_name='h2o_feet',
                            data_frame_tag_columns=['location'])

# Write with asynchronous client
### Data are writes in an asynchronous HTTP request.

In [12]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import ASYNCHRONOUS

client = InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api(write_options=ASYNCHRONOUS)

_point1 = Point("my_measurement").tag("location", "Prague").field("temperature", 25.3)
_point2 = Point("my_measurement").tag("location", "New York").field("temperature", 24.3)

async_result = write_api.write(bucket=bucket, record=[_point1, _point2])
async_result.get()

client.close()

# Write with synchronous client
### Data are writes in a synchronous HTTP request.

In [13]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

client = InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api(write_options=SYNCHRONOUS)

_point1 = Point("my_measurement").tag("location", "Prague").field("temperature", 25.3)
_point2 = Point("my_measurement").tag("location", "New York").field("temperature", 24.3)

write_api.write(bucket=bucket, record=[_point1, _point2])

client.close()

# Get Buckets

In [30]:
import influxdb_client.client.bucket_api as fluxdb

client = InfluxDBClient(url=url, token=token, org=org)
buckets_api = fluxdb.BucketsApi(client)

# All of buckets
all_buckets = buckets_api.find_buckets()

In [31]:
help(buckets_api)

Help on BucketsApi in module influxdb_client.client.bucket_api object:

class BucketsApi(builtins.object)
 |  BucketsApi(influxdb_client)
 |  
 |  Implementation for '/api/v2/buckets' endpoint.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, influxdb_client)
 |      Initialize defaults.
 |  
 |  create_bucket(self, bucket=None, bucket_name=None, org_id=None, retention_rules=None, description=None) -> influxdb_client.domain.bucket.Bucket
 |      Create a bucket.
 |      
 |      :param Bucket bucket: bucket to create (required)
 |      :param bucket_name: bucket name
 |      :param description: bucket description
 |      :param org_id: org_id
 |      :param bucket_name: bucket name
 |      :param retention_rules: retention rules array or single BucketRetentionRules
 |      :return: Bucket
 |               If the method is called asynchronously,
 |               returns the request thread.
 |  
 |  delete_bucket(self, bucket)
 |      Delete a bucket.
 |      
 |      :param bucket

# Read with Query

In [27]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

client = InfluxDBClient(url=url, token=token, org=org)

write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

p = Point("my_measurement").tag("location", "Prague").field("temperature", 25.3)

write_api.write(bucket=bucket, record=p)

## using Table structure
tables = query_api.query('from(bucket:' + "\"" + bucket + "\"" + ') |> range(start: -20m)')

for table in tables:
    print(table)
    for row in table.records:
        print (row.values)


## using csv library
csv_result = query_api.query_csv('from(bucket:' + "\"" + bucket + "\"" + ') |> range(start: -20m)')
val_count = 0
for row in csv_result:
    for cell in row:
        val_count += 1
print(csv_result)

FluxTable() columns: 9, records: 1
{'result': '_result', 'table': 0, '_start': datetime.datetime(2021, 7, 19, 14, 57, 44, 616869, tzinfo=tzutc()), '_stop': datetime.datetime(2021, 7, 19, 15, 17, 44, 616869, tzinfo=tzutc()), '_time': datetime.datetime(2021, 7, 19, 15, 9, 48, 829094, tzinfo=tzutc()), '_value': 1.0, '_field': 'water_level', '_measurement': 'h2o_feet', 'location': 'coyote_creek'}
FluxTable() columns: 9, records: 2
{'result': '_result', 'table': 1, '_start': datetime.datetime(2021, 7, 19, 14, 57, 44, 616869, tzinfo=tzutc()), '_stop': datetime.datetime(2021, 7, 19, 15, 17, 44, 616869, tzinfo=tzutc()), '_time': datetime.datetime(2021, 7, 19, 15, 9, 50, 881445, tzinfo=tzutc()), '_value': 24.3, '_field': 'temperature', '_measurement': 'my_measurement', 'location': 'New York'}
{'result': '_result', 'table': 1, '_start': datetime.datetime(2021, 7, 19, 14, 57, 44, 616869, tzinfo=tzutc()), '_stop': datetime.datetime(2021, 7, 19, 15, 17, 44, 616869, tzinfo=tzutc()), '_time': datetim